In [ ]:
import cv2
import numpy as np
import subprocess
import time
from PIL import Image
from io import BytesIO

def sell():
    press('sell_store.png',True,0.85)

    
def adb_command(command):
    result = subprocess.run(['adb', 'shell'] + command.split(), capture_output=True, text=True)
    return result.stdout.strip()

def capture_screenshot_with_scrcpy(output_path="screen.png"):
    try:
        result = subprocess.run(
            ["scrcpy", "--no-playback", "--output", output_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        if result.returncode == 0:
            print(f"Screenshot saved to {output_path}")
        else:
            print(f"Error during screenshot capture: {result.stderr.decode()}")
    except Exception as e:
        print(f"Error during screenshot capture: {e}")
        
def find_button_location(button_template_filename, capture, threshold):
    start = time.time()
    
    if capture:
        try:
            result = subprocess.run(
                ["adb", "exec-out", "screencap", "-p"], 
                stdout=subprocess.PIPE, stderr=subprocess.PIPE
            )
            if result.returncode == 0:
                screenshot_data = result.stdout
                with open("screen.png", "wb") as f:
                    f.write(screenshot_data)
            
        except Exception as e:
            print(f"Error during screenshot capture: {e}")

    

    end = time.time() - start
    seconds = (end % 60)
    print(f"capture {button_template_filename} time {seconds}")
    
    screenshot = cv2.imread('screen.png')

    if screenshot is None:
        print("Failed to load screenshot.")


    button_template = cv2.imread(button_template_filename)
    if button_template is None:
        print(f"Error loading button template: {button_template_filename}")
        return None


    screenshot_gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2GRAY)
    button_template_gray = cv2.cvtColor(button_template, cv2.COLOR_BGR2GRAY)

    result = cv2.matchTemplate(screenshot_gray, button_template_gray, cv2.TM_CCOEFF_NORMED)
    locations = np.where(result >= threshold)


    if len(locations[0]) > 0 and len(locations[1]) > 0:
        top_left = (locations[1][0], locations[0][0])
        button_width = button_template.shape[1]
        button_height = button_template.shape[0]
        center_x = top_left[0] + button_width // 2
        center_y = top_left[1] + button_height // 2
        return center_x, center_y
    else:
        return None

def press(button_template_filename, capture, threshold,i=0,c=0):
    
    time.sleep(0.006) 
    button_location = find_button_location(button_template_filename, capture, threshold)
    
    if button_location:
        center_x, center_y = button_location

        adb_command(f"input tap {center_x} {center_y}")
        i=1
        c=0
        i3=0
        c3=0
        if button_template_filename=='minus.png':
            for i in range(10):
                adb_command(f"input tap {center_x} {center_y}")
                
        if button_template_filename=='wh100.png':
            swipe_hori('scyte2.png', True, 0.49, dx=900, dy=400, hold_duration=1500, swipe_duration=400)

        
        if button_template_filename=='sell_close.png':
            i3,c3 = press('!.png', True, 0.70,i3,c3)
            sell()

        if button_template_filename=='sell_store.png':
            for i in range(10):
                press('log.png',True,0.8)
        
            press('wheat2.png',True,0.7)
            whatad = find_button_location('ad_dia.png', True, 0.9)
            if whatad:
                press('ad_close.png',False,0.8)
            else:
                press('ad1.png',True,0.7)
            press('close2.png',True,0.8)
            
        if button_template_filename=='log.png':
            press('wheat.png',True,0.85)
            press('most.png',True,0.85)
            
            press('minus.png',False,0.85)
            press('sell.png',False,0.85)
            
        if button_template_filename=='ad1.png':
            press('ad2.png',True,0.8)
        if button_template_filename=='!.png':
            cx=0
            ix=0

            while(cx != 2 ):
                press('sell_store.png', True, 0.8)
                ix,cx = press('sold.png',True, 0.8,ix,cx)


            press('close2.png',True,0.7)
    else:
        i=0
        c+=1
        if button_template_filename=='wh100.png':
            press('sell_close.png', True, 0.70)
        if button_template_filename=='wheat.png':
            press('close3.png',True,0.7)
            

    
    return i,c
    
def swipe(button_template_filename, capture, threshold, distance, angle_degrees, duration=300):

    button_location = find_button_location(button_template_filename, capture, threshold)
    
    if button_location:
        start_x, start_y = button_location

        angle_radians = math.radians(angle_degrees)
        
        dx = int(distance * math.cos(angle_radians))
        dy = int(-distance * math.sin(angle_radians))  
        
        end_x = start_x + dx
        end_y = start_y + dy

        print(f"Swiping from ({start_x}, {start_y}) to ({end_x}, {end_y}) over {duration}ms")

        adb_command(f"input swipe {start_x} {start_y} {end_x} {end_y} {duration}")
        

        
def swipe_hori(button_template_filename, capture, threshold, dx, dy, hold_duration=500, swipe_duration=650):
    button_location = find_button_location(button_template_filename, capture, threshold)
    
    if button_location:
        start_x, start_y = button_location
        end_x = start_x + dx
        end_y = start_y + dy


        adb_command(f"input swipe {start_x} {start_y} {end_x} {end_y} {swipe_duration}")

def swipe_test(button_template_filename, capture, threshold, dx, dy, hold_duration=500, swipe_duration=300):
    button_location = find_button_location(button_template_filename, capture, threshold)
    
    if button_location:
        start_x, start_y = button_location

        adb_command(f"input swipe {start_x} {start_y} {start_x} {start_y} {10}")
        adb_command(f"input swipe {start_x} {start_y} {start_x} {start_y+15} {10}")
        adb_command(f"input swipe {start_x} {start_y} {start_x} {start_y} {10}")
        adb_command(f"input swipe {start_x} {start_y} {start_x} {start_y-15} {10}")
        
        for i in range (2):
            end_x = start_x + 200*i
            end_y = start_y + 340*i
            if i !=0:
                start_x = tempx
                start_y = tempy
                adb_command(f"input swipe {start_x-35} {start_y-65} {end_x} {end_y} {swipe_duration}")
                
            tempx = end_x
            tempy = end_y






In [ ]:
count=1
while(1):
    i1=0
    c1=0
    
    i2=0
    c2=0

    i3=0
    c3=0
    
    while(c1 != 2 ):
        
        i1,c1 = press('wh100.png', True, 0.60,i1,c1)
    while(c2 != 2 ):
        i2,c2 = press('soil100_test.png', True, 0.85,i2,c2)
        swipe_test('seed_wh.png', True, 0.70, dx=0, dy=550, hold_duration=1500, swipe_duration=800)
        adb_command(f"input tap {1200} {300}")
        
    if count%6 ==0:
        i3,c3 = press('!.png', True, 0.70,i3,c3)
        sell()
    count +=1